In [1]:
import numpy as np
import cv2

In [2]:
digits = cv2.imread('./MasterOpenCV/images/digits.png', 0)

In [3]:
SZ = 20
def deskew(img):
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
    img = cv2.warpAffine(img, M, (SZ, SZ), flags=cv2.WARP_INVERSE_MAP | cv2.INTER_LINEAR)
    
    return img

winSize = (20,20)
blockSize = (10,10)
blockStride = (5,5)
cellSize = (10,10)
nbins = 9
derivAperture = 1
winSigma = -1.
histogramNormType = 0
L2HysThreshold = 0.2
gammaCorrection = 1
nlevels = 64
signedGradients = True
hog = cv2.HOGDescriptor(winSize,
                        blockSize,
                        blockStride,
                        cellSize,
                        nbins,
                        derivAperture,
                        winSigma,
                        histogramNormType,
                        L2HysThreshold,
                        gammaCorrection,
                        nlevels,
                        signedGradients)

In [4]:
cells = [np.hsplit(row,100) for row in np.vsplit(digits,50)]
train_cells = [ i[:50] for i in cells ]
test_cells = [ i[50:] for i in cells]
deskewed = [list(map(deskew, row)) for row in train_cells]
hogdata = [list(map(hog.compute,row)) for row in deskewed]
trainData = np.float32(hogdata).reshape(-1,81)
responses = np.repeat(np.arange(10),250)[:,np.newaxis]

deskewed = [list(map(deskew,row)) for row in test_cells]
hogdata = [list(map(hog.compute,row)) for row in deskewed]
testData = np.float32(hogdata).reshape(-1,81)

In [5]:
svm = cv2.ml.SVM_create()
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setType(cv2.ml.SVM_C_SVC)
svm.setC(1.19)
svm.setGamma(5.383)
svm.train(trainData, cv2.ml.ROW_SAMPLE, responses)

result = svm.predict(testData)[1]
mask = result==responses
correct = np.count_nonzero(mask)
print(correct*100.0/result.size)

97.36
